In [ ]:
# Module Imports
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import os
import cv2 as cv

In [ ]:
# creates alias for these classes
BaseOptions = mp.tasks.BaseOptions
GestureRecognizer = mp.tasks.vision.GestureRecognizer
GestureRecognizerOptions = mp.tasks.vision.GestureRecognizerOptions
GestureRecognizerResult = mp.tasks.vision.GestureRecognizerResult
VisionRunningMode = mp.tasks.vision.RunningMode
ClassifierOptions = mp.tasks.components.processors.ClassifierOptions

# absolute path to the gesture_recognizer TFlite file
model_path = os.getcwd() + "/exported_model/gesture_recognizer.task"
# base model recognizer options in common with other recognizers
base_opt= BaseOptions(model_asset_path=model_path)

# Use OpenCV’s VideoCapture to start capturing from the webcam.
cap = cv.VideoCapture(0)

In [ ]:
# Create a gesture recognizer instance with the live stream mode:
def print_result(result: GestureRecognizerResult, output_image: mp.Image, timestamp_ms: int):
    print('gesture recognition result: {}'.format(result))

# sets up classifier options (not implemented, just here for future use)
custom_classifier_options = ClassifierOptions() # https://developers.google.com/mediapipe/api/solutions/python/mp/tasks/components/processors/ClassifierOptions

# sets up gesture recognizer options
options = GestureRecognizerOptions(
    base_options=base_opt,
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result,
    min_hand_detection_confidence=0.5
    )

In [ ]:
frame_timestamp_ms = 0
with GestureRecognizer.create_from_options(options) as recognizer:
    # Create a loop to read the latest frame from the camera using VideoCapture#read()
    while cap.isOpened():
        # reads capture data and returns the frame
        ret, frame = cap.read()
        if not ret:
            print("Empty Frame")
            break
                
        # checks if escape key (q) has been pressed every 5 ms
        if cv.waitKey(5) & 0xff == ord('q'):
            break
        # Convert the frame received from OpenCV to a MediaPipe’s Image object.
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)

        frame_timestamp_ms += 1
        # Send live image data to perform gesture recognition.
        # The results are accessible via the `result_callback` provided in
        # the `GestureRecognizerOptions` object.
        # The gesture recognizer must be created with the live stream mode.
        recognizer.recognize_async(mp_image, frame_timestamp_ms)

# ends capture
cap.release()
cv.destroyAllWindows()

